## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Longyearbyen,SJ,78.2186,15.6401,24.64,68,75,8.05,broken clouds
1,1,Hualmay,PE,-11.0964,-77.6139,74.39,69,100,8.61,overcast clouds
2,2,Cedar City,US,37.6775,-113.0619,50.40,40,0,10.36,clear sky
3,3,Batemans Bay,AU,-35.7167,150.1833,67.19,83,100,6.31,overcast clouds
4,4,Khatanga,RU,71.9667,102.5000,-5.48,97,95,13.44,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp=float(input('What is the minimum temperature you would like for your trip?'))
max_temp=float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_city = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp)&(city_data_df["Max Temp"]>=min_temp)]

In [5]:
preferred_city

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Hualmay,PE,-11.0964,-77.6139,74.39,69,100,8.61,overcast clouds
6,6,Rikitea,PF,-23.1203,-134.9692,79.21,72,89,18.81,overcast clouds
7,7,Kapaa,US,22.0752,-159.3190,78.19,71,0,3.00,clear sky
9,9,Arraial Do Cabo,BR,-22.9661,-42.0278,71.46,88,100,4.61,overcast clouds
21,21,Paita,PE,-5.0892,-81.1144,82.96,61,47,14.25,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
697,697,Walvis Bay,NaN,-22.9575,14.5053,72.55,78,6,6.91,clear sky
698,698,Prainha,BR,-1.8000,-53.4800,78.96,89,77,3.09,broken clouds
700,700,Buenos Aires,AR,-34.6132,-58.3772,88.02,58,0,12.66,clear sky
701,701,Port Keats,AU,-14.2500,129.5500,76.96,92,92,6.38,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_city.count()

City ID        269
City           269
Country        266
Lat            269
Lng            269
Max Temp       269
Humidity       269
Cloudiness     269
Wind Speed     269
Description    269
dtype: int64

In [7]:
preferred_city.loc[preferred_city['Country'].isnull()]

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
93,93,Luderitz,NaN,-26.6481,15.1594,73.53,52,3,18.77,clear sky
473,473,Warmbad,NaN,-28.4500,18.7333,85.80,14,2,11.74,clear sky
697,697,Walvis Bay,NaN,-22.9575,14.5053,72.55,78,6,6.91,clear sky


In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_city.dropna(how='any',inplace=True)

C:\Users\sohra\anaconda3\envs\PythonData\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [9]:
clean_df=preferred_city

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Hualmay,PE,74.39,overcast clouds,-11.0964,-77.6139,
6,Rikitea,PF,79.21,overcast clouds,-23.1203,-134.9692,
7,Kapaa,US,78.19,clear sky,22.0752,-159.3190,
9,Arraial Do Cabo,BR,71.46,overcast clouds,-22.9661,-42.0278,
21,Paita,PE,82.96,scattered clouds,-5.0892,-81.1144,
22,Atuona,PF,78.35,clear sky,-9.8000,-139.0333,
24,Atar,MR,74.41,clear sky,20.5169,-13.0499,
26,Dingle,PH,75.96,overcast clouds,10.9995,122.6711,
30,Payo,ID,70.65,overcast clouds,-3.7508,103.6372,
31,Fonte Boa,BR,83.08,overcast clouds,-2.4959,-66.2662,


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    # 6e. Make request and retrieve the JSON data from the search.
    
    response=requests.get(base_url,params=params).json()  
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name']=response['results'][0]['name']
    except (IndexError):
        print('Missing hotel,skip this city')

Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city
Missing hotel,skip this city


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.loc[hotel_df['Hotel Name']==""]

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
31,Fonte Boa,BR,83.08,overcast clouds,-2.4959,-66.2662,
73,Muriti,TZ,74.08,overcast clouds,-1.9833,32.9167,
195,Murindo,CO,86.77,broken clouds,6.9848,-76.7544,
215,Raga,SS,76.91,scattered clouds,8.4596,25.6780,
219,Madimba,TZ,79.43,overcast clouds,-10.4000,40.3333,
281,Puro,PH,77.00,overcast clouds,12.4841,123.3827,
309,Amio,PH,75.42,overcast clouds,9.3571,122.9413,
338,Poum,NC,81.07,overcast clouds,-20.2333,164.0167,
396,Beloha,MG,72.86,scattered clouds,-25.1667,45.0500,
427,Mitu,CO,81.43,overcast clouds,1.1983,-70.1733,


In [13]:
hotel_df=hotel_df.drop([31,73,195,215,219,281,309,338,396,427,496,531,540,546,605,632])

In [14]:
# 8a. Create the output File (CSV)
output_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
fig

Figure(layout=FigureLayout(height='420px'))